# An $SU(3)\otimes SU(3)$ example

Start with a clean slate, import necessary stuff, and define some utility functions

In [97]:
%reset -f
import string
import numpy as np
from numpy import sqrt, cos, sin, pi
from scipy.linalg import norm, block_diag, expm, inv, eigh, det
import qiskit.quantum_info as qi
from qiskit.visualization import array_to_latex

import special_unitary_group as sug

def partial_trace(T, dims, trace_axes):
    """
    Compute partial trace for a subset of axes.
    :param T:  a numpy array of NxN
    :param dims: a tuple/list of integers of subspace dimensions, np.prod(dims) = N
    :param trace_axes: a tuple/list of subspace indices to partial trace over.
    :return: a numpy array of MxM where M = N/np.prod([dims[k] for k in trace_axes])

    Note: the order of dimensions are in the qiskit conversion,
    the Hilbert space the H_{K-1} X...X H_0.

    The limitation is the number of subspaces is no greater than 26 (by the
    number of letter used in einsum).  For the case where number of subspaces
    is greater than 26, use partial_trace_2, which is unlimited, but much slower.
    """

    K = len(dims)

    # build subscript string for einsum
    in_sub_left = list(string.ascii_lowercase[:K])  # in subscript left
    in_sub_right = list(string.ascii_uppercase[:K])  # in subscript right
    out_sub_left = list(string.ascii_lowercase[:K])  # out subscript left
    out_sub_right = list(string.ascii_uppercase[:K])  # out subscript right

    trace_dim = 1
    for k in trace_axes:
        out_sub_left.remove(in_sub_left[K-1-k])  # qiskit reverse
        out_sub_right.remove(in_sub_right[K-1-k])
        in_sub_right[K-1-k] = in_sub_left[K-1-k]
        trace_dim *= dims[k]

    subscript = "".join(in_sub_left + in_sub_right + ['-', '>'] + out_sub_left + out_sub_right)

    out_dim = round(np.prod(dims)/np.prod(trace_dim))
    dims_reversed = list(reversed(dims))
    return np.einsum(subscript, T.reshape(dims_reversed+dims_reversed)).reshape(out_dim, out_dim)

def process_density_matrix(rho):
    """
    take an input qi.DensityMatrix and returns single qudit density matrix (np array)
    :param rho: qiskit.quantum_info.DensityMatrix, density matrix of K qudits
    :return:
    """
    dims = rho.dims()
    K = len(dims)
    r1_list = []  # list of single qudit reduced density matrix
    eigen_list = []  # list of eigenvalues and eigenvectors of r1
    for k in range(K):
        trace_dims = list(range(K))
        trace_dims.remove(k)
        r1 = partial_trace(rho.data, dims, trace_dims)
        D, V = eigh(r1)
        V /= det(V)**(1/dims[k])  # make sure V in SU(N)
        r1_list.append(r1)
        eigen_list.append((D, V))

    return r1_list, eigen_list

def compute_SO2(v_a, v_b):
    norm_a = norm(v_a)
    norm_b = norm(v_b)
    if not np.isclose(norm_a, norm_b):
        raise ValueError("Norm v_a and v_b diff,\n"
                         f"    norm(v_a) = {norm_a},\n"
                         f"    norm(v_b) = {norm_b},\n"
                         f"    diff = {norm_a-norm_b}")
    denominator = norm_a**2

    cos_phi = min(1, max(-1, (v_a[0]*v_b[0] + v_a[1]*v_b[1])/denominator))
    sin_phi = min(1, max(-1, (v_b[0]*v_a[1]-v_a[0]*v_b[1])/denominator))

    # R = [[c, s], [-s, c]]
    return np.array([[cos_phi, sin_phi], [-sin_phi, cos_phi]])

def R2phi(R):
    return np.arctan2(R[0, 1], R[0, 0])

Now let's start with two qutrits.  Randomly generate $\rho^{(0)}$ and $U = U_1\otimes U_0$, then compute $\rho^{(1)} = U\rho^{(0)}U^\dagger$.  Note with python, the index starts from 0, while in the article, indices are starting from 1.

In [98]:
np.set_printoptions(linewidth=1000)
RNG = np.random.default_rng(12345)

n = 3
K = 2
dims = tuple([n]*K)

rho_a = qi.random_density_matrix(dims, seed=RNG)

U_list = [qi.random_unitary(n, seed=RNG).data for _ in range(K)]
U = 1
for i in range(len(U_list)):
    U = np.kron(U_list[i], U)

rho_b = rho_a.evolve(U)

rho_list = [rho_a, rho_b]

Compute the single qutrit reduced density matrices, and their eigendecompositions.

In [99]:
r1_lists = [list(), list()]
eigen_lists = [list(), list()]

for i in (0, 1):
    r1_lists[i], eigen_lists[i] = process_density_matrix(rho_list[i])

Check the agreement of eigenvalues.

In [100]:
for k in range(K):
    D_a = eigen_lists[0][k][0]
    D_b = eigen_lists[1][k][0]
    if not np.allclose(D_a-D_b, 0):
        raise ValueError(f"signle qudit eigen values in qudit-{k} differ"
                        f"D_a = {D_a}, D_b = {D_b}, Diff = {D_a-D_b}")
    print(f"D_{k}")
    display(array_to_latex(D_a))

D_0


<IPython.core.display.Latex object>

D_1


<IPython.core.display.Latex object>

Construct reference density matrices $\rho^{(r,a)}$ for $a = 0, 1$ as in Eq. (5).

In [101]:
ref_rho_list = []  # ref_rho_list is a list of np.array, not qi.DensityMatrix
for a in (0, 1):
    V = 1
    for k in range(K):
        V = np.kron(eigen_lists[a][k][1], V)
    ref_rho_list.append(V.conj().T @ rho_list[a].data @ V)
    print(f"ref_rho_list[{a}]")
    display(array_to_latex(ref_rho_list[-1], max_size=30))

ref_rho_list[0]


<IPython.core.display.Latex object>

ref_rho_list[1]


<IPython.core.display.Latex object>

Project $\rho^{(r,a)}$ to the basis as in Eq. (9)

In [102]:
P_list = []
for rr in ref_rho_list:
    P = sug.matrix_to_basis_projection(rr, dims).real
    rr1 = sug.basis_projection_to_matrix(P, dims)
    if not np.allclose(rr, rr1):
        raise ValueError("basis projection error")
    P_list.append(P)

if not np.allclose(P_list[0][0, :], P_list[1][0, :]):
    raise ValueError("P0[0, :] differ from P1[0, :]")
if not np.allclose(P_list[0][:, 0], P_list[1][:, 0]):
    raise ValueError("P0[:, 0] differ from P1[:, 0]")

Take the submatrix $P$ matrix as in Eq. (29)

In [103]:
p_mat_list = [P[1:, 1:] for P in P_list]
for p in p_mat_list:
    print("p")
    display(array_to_latex(p, max_size=30))

p


<IPython.core.display.Latex object>

p


<IPython.core.display.Latex object>

Now solve Eq. (30) and Eq. (31) for $R^{(0)}$ (python index starts with 0).  Because both qutrits are fully non-degenerate, both $R^{(a)}$ are block-diagonal as
$$R^{(a)} = R^{(a,12)}\oplus R^{(a,13)}\oplus R^{(a,23)}\oplus I_2.$$

In [104]:
R12 = compute_SO2(p_mat_list[0][0:2, -1], p_mat_list[1][0:2, -1])
R13 = compute_SO2(p_mat_list[0][2:4, -1], p_mat_list[1][2:4, -1])
R23 = compute_SO2(p_mat_list[0][4:6, -1], p_mat_list[1][4:6, -1])
phi12 = R2phi(R12)
phi13 = R2phi(R13)
phi23 = R2phi(R23)

R0 = block_diag(R12, R13, R23, np.eye(2))
display(array_to_latex(R0))

<IPython.core.display.Latex object>

Now solve for $\theta^{(0)}$ using Eq. (74).  As there are only two free $\theta^{(0)}$, we will use $\phi_{12}$ and $\phi_{13}$ to solve $\theta^{(0)}$.  Also, $f_{1,012,112} = 1$, $f_{2,012,112} = 0$, $f_{1,013,113} = 1/2$, and $f_{2,013,113} = \sqrt{3}/2$, thus
\begin{align}
\phi_{12} &= \theta^{(0)}_1 \\
\phi_{13} &= \frac12\theta^{(0)}_1 + \frac{\sqrt3}{2}\theta^{(0)}_2
\end{align}

In [105]:
theta1 = phi12
theta2 = (2*phi13-theta1)/sqrt(3)

Construct $\bar U_0$

In [106]:
su_n = sug.get_group_structure(n)
idx_bd = sug.get_index_bidict(n)

T1 = su_n.T[idx_bd[1]-1]
T2 = su_n.T[idx_bd[2]-1]
Ubar0 = expm(1j*(theta1*T1 + theta2*T2))
display(array_to_latex(Ubar0))

<IPython.core.display.Latex object>

From Eq. (30), $R^{(1)} = \left(P^{(1)}\right)^{-1}R^{(0)}P^{(0)}$.  And because $R^{(1)}$ is also block diagonal, we can solve for $\phi_{ij}$ and $\theta^{(1)}_l$ as before.

In [107]:
R1 = inv(p_mat_list[1])@R0@p_mat_list[0]
phi12 = R2phi(R1[0:2, 0:2])
phi13 = R2phi(R1[2:4, 2:4])
phi23 = R2phi(R1[4:6, 4:6])
theta1 = phi12
theta2 = (2*phi13-theta1)/sqrt(3)

Now construct $\bar U_1$ and verify
$$\rho^{(r, 1)} = \left(\bar U_1\otimes\bar U_0\right)\rho^{(r,0)}\left(\bar U_1\otimes\bar U_0\right)^\dagger.$$

In [108]:
Ubar1 = expm(1j*(theta1*T1+theta2*T2))
display(array_to_latex(Ubar1))
Ubar = np.kron(Ubar1, Ubar0)
print(np.allclose(ref_rho_list[1], Ubar@ref_rho_list[0]@Ubar.conj().T))

<IPython.core.display.Latex object>

True


In [109]:
%reset -f
import string
import numpy as np
from numpy import sqrt, cos, sin, pi
from scipy.linalg import norm, block_diag, expm, inv, eigh, det
import qiskit.quantum_info as qi
from qiskit.visualization import array_to_latex

import special_unitary_group as sug

np.set_printoptions(linewidth=1000)

def partial_trace(T, dims, trace_axes):
    """
    Compute partial trace for a subset of axes.
    :param T:  a numpy array of NxN
    :param dims: a tuple/list of integers of subspace dimensions, np.prod(dims) = N
    :param trace_axes: a tuple/list of subspace indices to partial trace over.
    :return: a numpy array of MxM where M = N/np.prod([dims[k] for k in trace_axes])

    Note: the order of dimensions are in the qiskit conversion,
    the Hilbert space the H_{K-1} X...X H_0.

    The limitation is the number of subspaces is no greater than 26 (by the
    number of letter used in einsum).  For the case where number of subspaces
    is greater than 26, use partial_trace_2, which is unlimited, but much slower.
    """

    K = len(dims)

    # build subscript string for einsum
    in_sub_left = list(string.ascii_lowercase[:K])  # in subscript left
    in_sub_right = list(string.ascii_uppercase[:K])  # in subscript right
    out_sub_left = list(string.ascii_lowercase[:K])  # out subscript left
    out_sub_right = list(string.ascii_uppercase[:K])  # out subscript right

    trace_dim = 1
    for k in trace_axes:
        out_sub_left.remove(in_sub_left[K-1-k])  # qiskit reverse
        out_sub_right.remove(in_sub_right[K-1-k])
        in_sub_right[K-1-k] = in_sub_left[K-1-k]
        trace_dim *= dims[k]

    subscript = "".join(in_sub_left + in_sub_right + ['-', '>'] + out_sub_left + out_sub_right)

    out_dim = round(np.prod(dims)/np.prod(trace_dim))
    dims_reversed = list(reversed(dims))
    return np.einsum(subscript, T.reshape(dims_reversed+dims_reversed)).reshape(out_dim, out_dim)

def process_density_matrix(rho):
    """
    take an input qi.DensityMatrix and returns single qudit density matrix (np array)
    :param rho: qiskit.quantum_info.DensityMatrix, density matrix of K qudits
    :return:
    """
    dims = rho.dims()
    K = len(dims)
    r1_list = []  # list of single qudit reduced density matrix
    eigen_list = []  # list of eigenvalues and eigenvectors of r1
    for k in range(K):
        trace_dims = list(range(K))
        trace_dims.remove(k)
        r1 = partial_trace(rho.data, dims, trace_dims)
        D, V = eigh(r1)
        V /= det(V)**(1/dims[k])  # make sure V in SU(N)
        r1_list.append(r1)
        eigen_list.append((D, V))

    return r1_list, eigen_list

def compute_SO2(v_a, v_b):
    norm_a = norm(v_a)
    norm_b = norm(v_b)
    if not np.isclose(norm_a, norm_b):
        raise ValueError("Norm v_a and v_b diff, "
                         f"norm(v_a) = {norm_a}, "
                         f"norm(v_b) = {norm_b}, "
                         f"diff = {norm_a-norm_b}")
    denominator = norm_a**2

    cos_phi = min(1, max(-1, (v_a[0]*v_b[0] + v_a[1]*v_b[1])/denominator))
    sin_phi = min(1, max(-1, (v_b[0]*v_a[1]-v_a[0]*v_b[1])/denominator))

    # R = [[c, s], [-s, c]]
    return np.array([[cos_phi, sin_phi], [-sin_phi, cos_phi]])

def R2phi(R):
    return np.arctan2(R[0, 1], R[0, 0])

n = 3

U_list = [
    np.array([[cos(pi/3), -sin(pi/3), 0],
              [sin(pi/3), cos(pi/3), 0],
              [0, 0, 1]]),
#    np.array([[cos(pi/4), 0, -sin(pi/4)],
#              [0, 1, 0],
#              [sin(pi/4), 0, cos(pi/4)]]),
    np.array([[1, 0, 0],
              [0, cos(pi/6), -sin(pi/6)],
              [0, sin(pi/6), cos(pi/6)]])
]

for i in range(len(U_list)):
    if not np.allclose(U_list[i]@U_list[i].conj().T, np.eye(3)):
        raise ValueError(f"U_list[{i}] is not unitary")


RNG = np.random.default_rng(12345)
U = 1
for i in range(len(U_list)):
#    U = np.kron(U_list[i], U)
    U = np.kron(qi.random_unitary(3, RNG).data, U)

rho_a = qi.random_density_matrix((3,3), seed=RNG)


rho_b = rho_a.evolve(U)

rho_list = [rho_a, rho_b]
for a in (0, 1):
    if not rho_list[a].is_valid():
        raise ValueError("Invalid density matrix")

r1_lists = [list(), list()]
eigen_lists = [list(), list()]

for i in (0, 1):
    rho = rho_list[i]
    r1_lists[i], eigen_lists[i] = process_density_matrix(rho)

K = len(r1_lists[0])

for k in range(K):
    D_a = eigen_lists[0][k][0]
    D_b = eigen_lists[1][k][0]
    if not np.allclose(D_a-D_b, 0):
        raise ValueError(f"signle qudit eigen values in qudit-{k} differ")
    print(f"D_{k}")
    display(array_to_latex(D_a))

# construct reference density matrices
ref_rho_list = []  # ref_rho_list is a list of np.array, not qi.DensityMatrix
for a in (0, 1):
    V = 1
    for k in range(K):
        V = np.kron(eigen_lists[a][k][1], V)
    ref_rho_list.append(V.conj().T @ rho_list[a].data @ V)
    print(f"ref_rho_list[{a}]")
    display(array_to_latex(ref_rho_list[-1], max_size=30))

p_mat_list = []
for ref_rho in ref_rho_list:
    p_mat_list.append(sug.matrix_to_basis_projection(ref_rho, (n, n))[1:,1:].real)
    print("P mat")
    display(array_to_latex(p_mat_list[-1], max_size=30))

# now solve for R0@P0@R1.T = P1
# because qudit 0 and qudit 1 are fully non-degenerate then
#  R0 = R0_12 (+) R0_13 (+) R0_13 + I_2
#  R1 = R1_12 (+) R1_13 (+) R1_13 + I_2
# (+) means matrix direct sum.
R12 = compute_SO2(p_mat_list[0][0:2, -1], p_mat_list[1][0:2, -1])
R13 = compute_SO2(p_mat_list[0][2:4, -1], p_mat_list[1][2:4, -1])
R23 = compute_SO2(p_mat_list[0][4:6, -1], p_mat_list[1][4:6, -1])
phi12 = R2phi(R12)
phi13 = R2phi(R13)
phi23 = R2phi(R23)

R0 = block_diag(R12, R13, R23, np.eye(2))
display(array_to_latex(R0))

# use ij = 12, 13 to solve Eq. (74)
# phi12 = theta_1*f_1,012,112 + theta_2*f_2,012,112
# phi13 = theta_1*f_1,013,113 + theta_2*f_2,013,113
# f1,012,112 = 1
# f2,012,112 = 0
# f1,013,113 = 1/2
# f2,013,113 = sqrt(3)/2
theta1 = phi12
theta2 = (2*phi13-theta1)/sqrt(3)
T1 = np.array([[0.5, 0, 0], [0, -0.5, 0], [0, 0, 0]])
T2 = np.array([[1/sqrt(12), 0, 0], [0, 1/sqrt(12), 0], [0, 0, -1/sqrt(3)]])
Ubar0 = expm(1j*(theta1*T1 + theta2*T2))
display(array_to_latex(Ubar0))

R1 = inv(p_mat_list[1])@R0@p_mat_list[0]
phi12 = R2phi(R1[0:2, 0:2])
phi13 = R2phi(R1[2:4, 2:4])
phi23 = R2phi(R1[4:6, 4:6])
theta1 = phi12
theta2 = (2*phi13-theta1)/sqrt(3)
Ubar1 = expm(1j*(theta1*T1+theta2*T2))
display(array_to_latex(Ubar1))
Ubar = np.kron(Ubar1, Ubar0)
print(np.allclose(ref_rho_list[1], Ubar@ref_rho_list[0]@Ubar.conj().T))

D_0


<IPython.core.display.Latex object>

D_1


<IPython.core.display.Latex object>

ref_rho_list[0]


...

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

True
